In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

# a)

In [15]:
def LR(X, T):
    """
    Retorna os coeficientes de um modelo de regressao linear
    
    Params:
        - X: dados de entrada
        - T: dados esperados
    """
    # criar uma vetor coluna para coeficiente w0
    uns = np.ones(len(X)).reshape(len(X),1)    
    X_ = np.concatenate([uns,X], axis=1)
    
    A = np.dot(X_.T,X_)
    B = np.dot(X_.T, T)
    A = np.linalg.inv(A)
    W = np.dot(A,B)
    return W

def predict(coef, X):
    """
    Retorna as predições de um modelo de regressao linear
    """
    uns = np.ones(len(X)).reshape(len(X),1)    
    X_ = np.concatenate([uns,X], axis=1)
    
    y_hat = np.dot(X_,coef)
    return y_hat

In [2]:
f = open("auto-mpg.data")
# le todas as linhas do arquivo
linhas = f.readlines()

In [3]:
# processamento dos dados
auto_data = []
for l in linhas:
    data = []
    # separa as linhas
    numeros_string = l.split('\t')
    for s in numeros_string[0].split("  "):
        s_strip = s.strip()
        if len(s_strip)>0:
            data.append(s_strip)
    data.append(numeros_string[1].strip())
    auto_data.append(data)

In [4]:
auto_data = np.array(auto_data)
df_auto_data = pd.DataFrame(auto_data)
df_auto_data.head()

,0,1,2,3,4,5,6,7,8
0,18.0,8,307.0,130.0,3504.,12.0,70,1,"""chevrolet chevelle malibu"""
1,15.0,8,350.0,165.0,3693.,11.5,70,1,"""buick skylark 320"""
2,18.0,8,318.0,150.0,3436.,11.0,70,1,"""plymouth satellite"""
3,16.0,8,304.0,150.0,3433.,12.0,70,1,"""amc rebel sst"""
4,17.0,8,302.0,140.0,3449.,10.5,70,1,"""ford torino"""


In [6]:
# remove a última coluna
df_auto_data = df_auto_data.drop(columns=8)

**Porque remover a última coluna?**:

A regressão linear só pode ser feita em valores numéricos e a última coluna é uma string.

In [7]:
# remove linhas que contem o valor "?", que representam dados faltantes
columns = df_auto_data.columns
for c in columns:
    df_auto_data = df_auto_data[df_auto_data[c]!='?']

In [8]:
# convert os valores para float
X = df_auto_data.values.astype('float')

## separa os dados em treino e teste
# dados de treino
X_train = X[:150, 1:]
y_train = X[:150, 0]
# dados de teste
X_test = X[150:, 1:]
y_test = X[150:, 0]

In [13]:
# obtencao dos coeficientes
coeficientes = LR(X_train, y_train)
# predicao dos valores
y_pred = predict(coeficientes,X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print ("rmse", rmse)

rmse 6.247006759639527


# b)

In [19]:
def RSS(orig_coef_, y_real, X):
    """
    Calcula o RSS de modelo de regressão linear
    Params:
        - orig_coef: coeficientes do modelo
        - y_real: y esperado
        - X: base de dados
    
    return:
        - sum_eq: soma dos quadrados dos erros
    
    """
    # obtem predições do modelo
    y_hat = predict(orig_coef_, X)
    # obtem os quadrodos dos erros
    erros_quadrados = np.array([(y_hat[i] - y_real[i])**2 for i in range(len(X))])
    sum_eq = erros_quadrados.sum()
    return sum_eq

In [20]:
def RSS_0(orig_coef_, y_real, X, indice_attr):
    """
    Calcula o RSS_0 de modelo de regressão linear
    Params:
        - orig_coef: coeficientes do modelo
        - y_real: y esperado
        - X: base de dados
    
    return:
        - sum_eq: soma dos quadrados dos erros
    
    """
    coef_ = orig_coef_.copy()
    # zera o coeficiente indicado
    coef_[indice_attr+1] = 0
    y_hat = predict(coef_, X)
    erros_quadrados = np.array([(y_hat[i] - y_real[i])**2 for i in range(len(X))])
    sum_eq = erros_quadrados.sum()
    return sum_eq

In [21]:
def F(rss, rss_0, n, q, N):
    """
    Obtem o F
    
    Params:
        - rss: valor do RSS do modelo
        - rss_0: valor do RSS_0 do modelo
        - n: número total de atributos
        - q: número de atributos na estimativa
        - N: número de amostras
    """
    numerador = (rss_0 - rss)/(n-q)
    denominador = (rss/(N-n-1))
    f = numerador/denominador
    return f

In [24]:
rss = RSS(coeficientes, y_train, X_train)
RSS_0s = [RSS_0(coeficientes, y_train, X_train, i) for i in range(7)]
Fs = [ F(rss, RSS_0s[i],7,6,len(X_train[:15])) for i in range(7)]

In [26]:
F142 = 3.908

In [27]:
# verifica quais atributos não passam no valor estabelecido
i = 1
for f in Fs:
    if(f<F142): print (f,i)
    i+=1

2.5405267435592926 1
0.13433013742832733 2
0.1558884279880601 5


$F_1 = 2.54$, $F_2=0.13$ e $F_5 = 0.15$. Logo, $F_1$,$F_2$ e $F_5$ podem ser removidos

In [27]:
novo_coef = orig_coef_.copy()
novo_coef[0] = 0
novo_coef[1] = 0
novo_coef[4] = 0
f_xN = [reg.intercept_ + ((novo_coef * X_test[i])).sum() for i in range(len(X_test))]

In [28]:
novo_rmse = np.sqrt(mean_squared_error(y_test, f_xN))
novo_rmse

5.539475549611815

# c)

**Comparação dos resultados**

O RMSE do modelo quando utilizando todas as variáveis era de $\approx 6.24$, enquanto que o RMSE do modelo ao se remover as variáveis (1, 2 e 5) após a aplicação do teste F de Snedecor é de $\approx 5.53$. Em outras palavras, a presença de tais variáveis durante a utilização do modelo de regressão linear prejudica o seu desempenho.